import packages & selct dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import spatial

import mosaic_topog.utilities as mostop
import mosaic_topog.flsyst as flsyst
import mosaic_topog.show as show
import mosaic_topog.calc as calc

import os

fold_path = 'C:\\Users\\schle\Documents\\GitHub\\Incubator-2022-Geometry-of-Color\\CSV data all OCT locs Jan 2022\\'

subject = np.asarray(['AO001R', 'AO008R'])
angle = np.asarray(['temporal', 'nasal', 'superior', 'inferior'])
eccentricity = np.asarray([1.5, 4, 10])

coord_fltype = '.csv'
img_fltype = '.png'

filetypes = [coord_fltype,img_fltype]

conetype_ids = ['all','L','M','S','unclassed']

conetype_colors = ['y','r','g','b','k']
conetype_sizes = [10, 10, 10, 30]

bin_width = 5

num_mc = 100 # number of monte carlo mosaics to generate

categories = [subject, angle, eccentricity]

load in coordinate data and images

In [ ]:
[data, subj_vect, ang_vect, ecc_vect, contyp_vect, fltyp_vect, mos_vect] = flsyst.getConeData(fold_path,subject,angle,eccentricity,conetype_ids,filetypes)

plot cones on the image 

In [ ]:
img_inds = np.nonzero(fltyp_vect==1)[0]

mosaics = np.unique(mos_vect)

for mos in mosaics:
    coords = []
    this_mos = np.nonzero(mos_vect==mos)[0]
    img = data[np.intersect1d(this_mos,np.nonzero(fltyp_vect==1)[0])[0]]
    
    for conetype,string in enumerate(conetype_ids):
        
        this_contyp = np.nonzero(contyp_vect==conetype)[0]
        coords.append(data[np.intersect1d(this_mos,this_contyp)[0]])
        
    show.plotOnROI(img, coords, conetype_ids, conetype_colors,10)
    

calculate the distances between all cones in each mosaic

In [ ]:
for mos in mosaics:
    
    